# Data Creation

I have been trying to build models on this dataset, but due to the size I felt it would be more comfortable to break it up into two parts. In this notebook I will create datasets with additional variables, and I will later use them in a separate notebook to create predictions and submit. 

The second part can be found here - [https://www.kaggle.com/craniket/g-research-lgbm-basic-model](http://)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gresearch_crypto
import matplotlib.pyplot as plt
env = gresearch_crypto.make_env()
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.model_selection import train_test_split

Here we define a function to reduce the size of the data.

In [ ]:
def reduce_mem_usage(train_data):
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in train_data.columns:
        col_type = train_data[col].dtype
        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return train_data

In [ ]:
data_dir = "/kaggle/input/g-research-crypto-forecasting/"
crypto_df = pd.read_csv(data_dir+"train.csv")

In [ ]:
crypto_df = reduce_mem_usage(crypto_df)

In [ ]:
crypto_df.head()

In [ ]:
crypto_name = pd.read_csv(data_dir+"asset_details.csv")
crypto_name

In [ ]:
#crypto_df.loc[crypto_df.Asset_ID == 2].plot(x = "timestamp",y = "Target")

In [ ]:
plot_df = crypto_df.merge(crypto_name,on = "Asset_ID")
plot_df = plot_df.sort_values(by = ['Asset_Name','timestamp'])
plot_df.head()

In [ ]:
assets = plot_df.Asset_Name.unique()
assets

#  Adding features

In [ ]:
plot_df['change'] = plot_df['Close']/plot_df['Open']
plot_df['variation'] = plot_df['High']/plot_df['Low']
plot_df['change_diff'] = plot_df['Close'] - plot_df['Open']
plot_df['variation_diff'] = plot_df['High'] - plot_df['Low']
def hlco_ratio(df): return (df['High'] - df['Low'])/(df['Close']-df['Open'])
def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']
plot_df['Upper_Shadow'] = upper_shadow(plot_df)
plot_df['Lower_Shadow'] = lower_shadow(plot_df)
plot_df['hlco_ration'] = hlco_ratio(plot_df)

In [ ]:
del crypto_df

In [ ]:
#def reindex(df):
#    df = df.reindex(range(df.index[0],df.index[-1]+60,60),method='nearest')
#    return df
plot_df = plot_df.sort_values(by = ["Asset_ID",'timestamp'])
#plot_df = plot_df.set_index(['timestamp',"Asset_ID"])
#plot_df = reindex(plot_df)

#  Re-indexing

The timestamps are not continuous for each Crypto. We need to reindex each individually.

In [ ]:
plot_df2 = pd.DataFrame()
def reindex(df):
    df = df.reindex(range(df.index[0],df.index[-1]+60,60),method='nearest')
    df = df.assign(filt = (df.index - df.index[0])/(df.index[-1]+60 - df.index[0]))
    return df
for asset_id,asset_name in enumerate(assets):
    df = plot_df.loc[plot_df["Asset_ID"] == asset_id]
    df = df.set_index(['timestamp'])
    df = reindex(df)
    plot_df2 = plot_df2.append(df)
plot_df2.head()

In [ ]:
plot_df2.shape

In [ ]:
plot_df2_train = plot_df2.loc[plot_df2["filt"]<=0.7]
plot_df2_test = plot_df2.loc[plot_df2["filt"]>0.7]
[plot_df2_train.shape,plot_df2_test.shape]

In [ ]:
#del plot_df2
[x_train,y_train,x_test,y_test] = [plot_df2_train.drop(["Target","Asset_Name","Weight"],axis=1),
                                  plot_df2_train.Target,
                                  plot_df2_test.drop(["Target","Asset_Name","Weight"],axis=1),
                                  plot_df2_test.Target]


[x_train.shape,y_train.shape,x_test.shape,y_test.shape] 


In [ ]:
weight_train,weight_test = plot_df2_train.Weight,plot_df2_test.Weight


In [ ]:
x_train.to_csv("x_train.csv")

In [ ]:
x_test.to_csv("x_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")
weight_train.to_csv("weight_train.csv")
weight_test.to_csv("weight_test.csv")

